In [1]:
import pandas as pd
import time
import datetime

# 🕒 Start timer
start_time = time.time()
last_timestamp = start_time
print("Clearance Transfer Start:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# -----------------------------
# Read CSV safely
# -----------------------------


df = pd.read_csv('clearance_transfer_input_windsor.csv', sep=',')  # use ',' if comma-separated

# Clean column headers

df.columns = df.columns.str.strip().str.replace('\n','').str.replace('\r','')


# This will store units sold per SKU per store
units_sold_db = {}  # key = SKU, value = dict of {store: sold_items}

sales_columns = {
    'Dougall': 'Dougall',
    'Huron': 'Huron',
    'Tecumseh': 'Tecumseh',
    'Walker': 'Walker',
    'Wyandotte': 'Wyandotte'
}

inv_columns = {
    'Dougall': 'DougallInv',
    'Huron': 'HuronInv',
    'Tecumseh': 'TecumsehInv',
    'Walker': 'WalkerInv',
    'Wyandotte': 'WyandotteInv'
}


# Ensure numeric columns are integers
for col in list(sales_columns.values()) + list(inv_columns.values()):
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Save initial sales for fallback
initial_sales = df[list(sales_columns.values())].copy()


# function to calculate units sold per iteration
# Initialize the database at the start
units_sold_db = {}  # key = SKU, value = dict of {store: sold_items}
    
# Transfer logic starts
# -----------------------------
transfers = []

updated_available_numbers = {store: [] for store in sales_columns.keys()}


# Function to calculate units sold for a single store (unchanged)
def calculate_units_sold(index, store):
    


    """
    Calculate units sold per SKU and store.
    """
#    print("Shank in calculate_units_sold function")
    sold_items = 0
    sales = df.loc[index, sales_columns[store]]
    inventory = df.loc[index, inv_columns[store]]
#    print("SHANK in CALCULATE_UNITS_SOLD: ")
#    print("SHANK QWE inventory: ", inventory)
#    print("SHANK QWE sales: ", sales)
    
    if inventory > sales:
        for i in range(inventory, sales, -1):
            sold_items += 1
    else:
        if inventory == sales:
            sold_items = 0
#    print("Shank sold_items: ", sold_items)
    return sold_items


def get_units_sold_array(index):
    
    """ Captures initial units sold
    Calculate units sold for all stores for a given SKU
    and store it in an array of tuples: (store_name, sold_items)
    """
#    print("Shank in get_units_sold_array function")
    units_sold_array = []
    for store in sales_columns.keys():
        sold_items = calculate_units_sold(index, store)  # recalculate per iteration
        units_sold_array.append((store, sold_items))
    return units_sold_array


# Determine DONOR Store
def select_donor(index):
#    print("Shank Entered select_donor function")
    
    """
    Select a single donor store for a given SKU.
    - Calculates units sold for each store dynamically.
    - Finds the store with the largest difference: inventory - sold_items.
    - Sets a flag to indicate whether a donor was found.
    - If no eligible donor, calls No_donor().
    Returns:
        donor (str or None): selected donor store
        donor_found (bool): True if donor found, False if fallback
    """
    donor = None
    donor_found = False
    donor_no_sale = False
    transfer_possible = True
#    max_excess = -float('inf')  # start very small to ensure any positive excess is considered
    max_excess = initial_sales.loc[index].max()  # max initial sale among all stores for this SKU
    

    # Calculate units sold array for all stores
    units_sold_array = get_units_sold_array(index)
#    print(f"Shank | SKU={df.loc[index, 'SKU']} | Units Sold Array={units_sold_array} | Max Excess={max_excess}")
    print(f"Shank | SKU={df.loc[index, 'SKU']} | Units Sold Array={units_sold_array}")
    
    # Determine donor based on max (inventory - sold_items)
    print("Shank about to enter no_sale function")
    no_sale(index)
    print("Shank post no_sale function", sales_made)
#    for store, sold in units_sold_array:
    if sales_made:
        print("SHANK sales_made for this sku: ", index)
        while transfer_possible:
            print("SHANK Start of WHILE")
            for store, inv_col in inv_columns.items():
                inventory = df.loc[index, inv_columns[store]]
#                print("Shank Store BF", store)
#                print("Shank Inventory BF", inventory)
#               print("Shank sold BF", sold)
                excess = 0
                donor_no_sale = False 
#               print("Shank excess", excess)
#               print("Shank donor_no_sale", donor_no_sale)
#               print("SHANK Calculate inventory here")
#               print("SHANK add a while loop for following functionality until Almost_equal flag is set)
#                print("calculate units_sold_array here ")
                sold = calculate_units_sold(index, store)
                print("SHANK Store", store)
                print("SHANK inventory: ", inventory)
                print("Shank Units sold AF: ", sold)

                if (inventory > 0) and (sold < inventory):
                    print("SHANK Trigger logic #1")
                    print("Shank donor store: ", store)
                    print("SHANK donor inventory: ", inventory)
                    print("Shank donor sold: ", sold)
                    transfer_log(index, store)
                    print("Shank 1 unit moved")
                    sold = calculate_units_sold(index, store)
                    break
#                   print("SHANK add and subtract inventory and sales")
#                elif( inventory == 0) and (sold < inventory):
#                    print("SHANK elif trigger transfer logic")
#                else:
#                    print("go to next sku")
            
#            print("SHANK 1 UNIT MOVED")
            print("Post transfer calculations for another check")
            if do_not_transfer == True:
                break
            transfer_possible = False
            for store, inv_col in inv_columns.items():
                post_inventory = df.loc[index, inv_columns[store]]
                post_sold = calculate_units_sold(index, store)
                post_sales = df.loc[index, sales_columns[store]]
                print("SHANK Store: ", store)
                print("SHANK post_sales: ", post_sales)
                print("SHANK post_sold: ", post_sold)
                print("SHANK post_inventory: ", post_inventory)
                if post_inventory > 0:
                    if (post_inventory - post_sold > 1):
                         print ("Shank setting transfer_possible to True")
                         transfer_possible = True
                         break
                    elif (post_inventory > post_sold ):
                         print ("SHANK inventory - sold = 1")
                         transfer_possible = True
                         break
                    else:
                         print ("Shank setting transfer_possible to False")
                         transfer_possible = False
    
            
    # Fallback if no donor found
    if donor is None:
        donor = None
        donor_found = False
    return donor, donor_found



# Determine Receiver Store
def select_receiver(index):
    
    """
    Select a receiver store for a given SKU.
    - Calculates units sold for all stores dynamically.
    - Chooses the store with the **highest units sold**.
    - Handles both positive and negative sales correctly.
    Returns:
        receiver (str): store selected to receive units
    """
    global receiver_sold
#    print("Shank in select_receiver function")
    # Calculate units sold array for all stores
    units_sold_array = get_units_sold_array(index)

#    print("Shank before selecting receiver")
    # Select store with maximum units sold
    receiver = max(units_sold_array, key=lambda x: x[1])[0]  # x[1] is sold_items
#    print("Shank post receiver selection", receiver)

    return receiver

def fallback_transfer(donor):
    pass
    
def no_sale(index):
#    print("Shank in no_sale function")
    global sales_made
    sales_made = True
#    print ("SHANK START sales_made: ", sales_made)
    for store, inv_col in inv_columns.items():
        inventory = df.loc[index, inv_columns[store]]
        sold_units = calculate_units_sold(index, store)
##        print("Shank Store AF", store)
#        print("Shank Inventory AF", inventory)
#        print("Shank sold AF", sold_units)
        if ( sold_units == 0):
#            print("SHank set sales_made false")
            sales_made = False
        else:
 #           print ("Shank set sales_made true")
            sales_made = True
            break
 #   print ("Shank AF sales_made: ", sales_made)
    if (not sales_made):
#        print("SHANK no sale transfer logic")
        no_sale_donor = None
        for store, inv_col in inv_columns.items():
            no_sale_inv = df.loc[index, inv_columns[store]]
            if (no_sale_inv > 0):
#                print ("SHANK Store: ", store)
#                print ("SHANK inventory: ", no_sale_inv )
                no_sale_donor = store
                no_sale_receiver = "Huron"
                if (no_sale_donor == no_sale_receiver):
#                    print("SHANK receiver = donor = Huron")
                    no_sale_receiver = "Tecumseh"
                df.loc[index, inv_columns[no_sale_donor]] -= 1
                df.loc[index, sales_columns[no_sale_receiver]] += 1
                
                transfers.append({
                    'SKU': df.loc[index, 'SKU'],
                    'Product Name': df.loc[index, 'Product Name'],
                    'Category': df.loc[index, 'Category'],
                    'From Store': no_sale_donor,
                    'To Store': no_sale_receiver,
                    'Amount': 1
                })
#                print("SHANK no sale Transfer Written")
                break
#    print("end of no_sale function sales_made: ", sales_made)
     
    
def transfer_log(index, store):
    """
    Executes a single transfer from donor to receiver and logs it.
    Uses your existing `transfers` list.
    """
    print("TRIP in transfer_log")
    global do_not_transfer
    do_not_transfer = False
    global transfers  # so that appends affect the main list
    donor = store
    # 1️⃣ Determine the receiver dynamically
    
    receiver = select_receiver(index)
    print("Shank receiver: -",receiver)
    receiver_units_sold = calculate_units_sold(index, receiver)
    donor_units_sold = calculate_units_sold(index, donor)
    print("Shank DONOR: -",donor)
    print("SHASHANK donor_units_sold: ", donor_units_sold)
    print("SHASHANK recevier_units_sold: ", receiver_units_sold)
    donor_sales = df.loc[index, sales_columns[donor]]
    donor_inventory = df.loc[index, inv_columns[donor]]
    
    
    # 2️⃣ Skip if donor and receiver are the same
    if (donor == receiver):
#        print("SHANK RETURN")
        do_not_transfer = True
        return
#    if (donor_inventory - donor_sales < 1):
#        return
#    if (donor_inventory < donor_sales ):
#        return
    
#    if (donor_inventory - donor_sold > 1):
#    print("TRIP Before Loop")
    if receiver == "Dougall" and receiver_units_sold == 0:
        receiver = "Huron"
#        print("@@@@@@@@@@@@@")
        if donor == receiver:
#            print("#############")
            receiver = "Tecumseh"
    # 3️⃣ Update inventory and sales
    
    
    print("Shashank donor FINAL : ", donor)
    print("Shashank receiver FINAL : ", receiver)
    current_sales = df.loc[index, sales_columns[receiver]]
    df.loc[index, inv_columns[donor]] -= 1
    df.loc[index, sales_columns[donor]] -= 1
#********************* new code**************
    df.loc[index, inv_columns[receiver]] +=1
    df.loc[index, sales_columns[receiver]] +=1
#********************* new code***************    
    print("Updated donor inventory:", df.loc[index, inv_columns[donor]])
    print("Updated donor Sales:", df.loc[index, sales_columns[donor]])
    print("Updated receiver Sales:", df.loc[index, sales_columns[receiver]])
    print("Updated receiver inventory:", df.loc[index, inv_columns[receiver]])
#    print ("SHANK Donor: ", donor)
#    print ("SHANK Donor Sales: ", donor_sales)
#        if donor_sales >= 0:
#            df.loc[index, sales_columns[donor]] -= 1
#    print("TRIP current_Sales", current_sales)
#    if current_sales > 0:
#         print(" TRIP #1")
#        df.loc[index, sales_columns[receiver]] -= 1
#        df.loc[index, inv_columns[receiver]] += 1
#    elif current_sales <= 0:
#         print("TRIP #2!")
#        df.loc[index, inv_columns[receiver]] += 1
#        df.loc[index, sales_columns[receiver]] += 1 

#    elif current_sales == 0:
#        if receiver == "Dougall":
#           print("TRIP 2")
#           dougall_sold = calculate_units_sold(index, "Dougall")
#           if dougall_sold == 0:
#               print("TRIP 3")
#               print("**********TRANSFER DEBUG INFO*************")
#               print("SKU: ", df.loc[index, 'SKU'])
#               print("Donor: ", donor)
#               print("Receiver: ", receiver)
#               print("Items transfered: 10")
#               print("***********TRANSFER INFO END**************")
               
#               transfers.append({
#                   'SKU': df.loc[index, 'SKU'],
#                   'Product Name': df.loc[index, 'Product Name'],
#                   'Category': df.loc[index, 'Category'],
#                  'From Store': donor,
#                   'To Store': receiver,
#                   'Amount': 9
#               })

    # 4️⃣ Log the transfer
    transfers.append({
            'SKU': df.loc[index, 'SKU'],
            'Product Name': df.loc[index, 'Product Name'],
            'Category': df.loc[index, 'Category'],
            'From Store': donor,
            'To Store': receiver,
            'Amount': 1
        })

def check_almost_equal():
    pass

# -----------------------------
# Transfer logic loop start
# -----------------------------

# -----------------------------
# Main module before transfer logic
# -----------------------------



# print("Entering Main Loop")
status_interval = 15  # seconds
last_status_time = time.time()

for index in df.index:  # Loop through each SKU
    sku = df.loc[index, 'SKU']
    product_name = df.loc[index, 'Product Name']
    category = df.loc[index, 'Category']
    current_time = time.time()
    if current_time - last_status_time >= status_interval:
        elapsed = int(current_time - start_time)
        print(f"[{elapsed}s] Processing SKU: {sku} ({product_name})")
        last_status_time = current_time
   # 1️⃣ Select donor and receiver dynamically
    print("shank entering donor and receiver loop")
    print("SHANK LOOP START FOR SKU: ", sku)
    donor, donor_found = select_donor(index)
#    receiver = select_receiver(index)
    # 2️⃣ Perform transfer if donor found and donor != receiver
#    print("shank checking if donor_found and donor = receiver")
#    if donor_found and donor != receiver:
#        print("shank transfering unit from donor to receiver")
        # Transfer 1 unit
#        df.loc[index, inv_columns[donor]] -= 1       # Subtract 1 from donor inventory
#        df.loc[index, sales_columns[receiver]] += 1 # Add 1 to receiver sales

        # Log the transfer
#        transfers.append({
#            'SKU': sku,
#            'Product Name': product_name,
#            'Category': category,
#            'From Store': donor,
#            'To Store': receiver,
#            'Amount': 1
#        })

    # 3️⃣ If no donor found, call fallback function
#    elif not donor_found:
#        fallback_transfer(index, receiver)  # You will define this function later



# Create a new DataFrame from the transfers list
transfers_df = pd.DataFrame(transfers, columns=['SKU', 'Product Name', 'Category', 'From Store', 'To Store', 'Amount'])
#print("Shank created a df transfers_df")


# Save the result to a new CSV file
transfers_df.to_csv('clearance_transfers_output_windsor.csv', index=False)

# Store the updated available numbers at each store for each SKU
for _, row in df.iterrows():
        for store in sales_columns.keys():  # <-- use sales_columns.keys() instead of store_columns
            updated_available_numbers[store].append(row[inv_columns[store]])  # updated inventory

updated_available_df = pd.DataFrame(updated_available_numbers, columns=['Dougall', 'Huron', 'Tecumseh', 'Walker', 'Wyandotte'])

updated_available_df = pd.concat([
      df[['SKU', 'Product Name', 'Category', 'DougallInv', 'HuronInv', 'TecumsehInv', 'WalkerInv', 'WyandotteInv']],
      updated_available_df
], axis=1)

# Save the updated available numbers to a new CSV file
updated_available_df.to_csv('updated_available_numbers_windsor.csv', index=False)

end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")





Clearance Transfer Start: 2025-11-25 10:51:24
shank entering donor and receiver loop
SHANK LOOP START FOR SKU:  106906_28g___
Shank | SKU=106906_28g___ | Units Sold Array=[('Dougall', 0), ('Huron', 3), ('Tecumseh', 1), ('Walker', 1), ('Wyandotte', 0)]
Shank about to enter no_sale function
Shank post no_sale function True
SHANK sales_made for this sku:  0
SHANK Start of WHILE
SHANK Store Dougall
SHANK inventory:  2
Shank Units sold AF:  0
SHANK Trigger logic #1
Shank donor store:  Dougall
SHANK donor inventory:  2
Shank donor sold:  0
TRIP in transfer_log
Shank receiver: - Huron
Shank DONOR: - Dougall
SHASHANK donor_units_sold:  0
SHASHANK recevier_units_sold:  3
Shashank donor FINAL :  Dougall
Shashank receiver FINAL :  Huron
Updated donor inventory: 1
Updated donor Sales: 1
Updated receiver Sales: -1
Updated receiver inventory: 2
Shank 1 unit moved
Post transfer calculations for another check
SHANK Store:  Dougall
SHANK post_sales:  1
SHANK post_sold:  0
SHANK post_inventory:  1
SHANK

In [3]:
import csv

def condense_csv(input_file, output_file):
    data = {}

    # Read CSV file and store data in a dictionary
    with open(input_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Assuming the first row contains headers
        for row in reader:
            key = tuple(row[i] for i in [0, 3, 4])  # Columns 1, 4, and 5
            if key in data:
                data[key][5] = str(int(data[key][5]) + int(row[5]))  # Combine values in column 6
            else:
                data[key] = row

    # Write condensed data to a new CSV file
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        for row in data.values():
            writer.writerow(row)

# Example usage
input_file = 'clearance_transfers_output_windsor.csv'  # Replace with your CSV file path
output_file = 'Clearance_Transfers_Output_Windsor.csv'  # Replace with the desired output file path
condense_csv(input_file, output_file)



